### Notebook setup

In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

### Create Engine/Session

In [2]:
import sqlalchemy_cockroachdb

sqlalchemy_cockroachdb

<module 'sqlalchemy_cockroachdb' from '/opt/conda/lib/python3.9/site-packages/sqlalchemy_cockroachdb/__init__.py'>

<IPython.core.display.Javascript object>

In [3]:
sqlalchemy_cockroachdb.__version__

'1.4.3.dev0'

<IPython.core.display.Javascript object>

In [4]:
!pip show sqlalchemy-cockroachdb

Name: sqlalchemy-cockroachdb
Version: 1.4.3.dev0
Summary: CockroachDB dialect for SQLAlchemy
Home-page: https://github.com/cockroachdb/sqlalchemy-cockroachdb
Author: Cockroach Labs
Author-email: cockroach-db@googlegroups.com
License: http://www.apache.org/licenses/LICENSE-2.0
Location: /opt/conda/lib/python3.9/site-packages
Requires: SQLAlchemy
Required-by: 


<IPython.core.display.Javascript object>

In [5]:
import sqlalchemy as sa
import sqlalchemy.orm
from sqlalchemy.ext.asyncio import create_async_engine, AsyncSession


engine = create_async_engine(
    "cockroachdb+asyncpg://root@cockroach:26257/defaultdb", echo=True
)
engine

<IPython.core.display.Javascript object>

In [6]:
async_session = sa.orm.sessionmaker(engine, class_=AsyncSession)
async_session

sessionmaker(class_='AsyncSession', bind=<sqlalchemy.ext.asyncio.engine.AsyncEngine object at 0x7fbd01ef9300>, autoflush=True, expire_on_commit=True)

<IPython.core.display.Javascript object>

### Drop/create tables

In [7]:
from models import Base, User, Message

async with engine.begin() as conn:
    await conn.run_sync(Base.metadata.drop_all)

2021-11-09 16:13:00,050 INFO sqlalchemy.engine.Engine select current_schema()
2021-11-09 16:13:00,051 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-11-09 16:13:00,057 INFO sqlalchemy.engine.Engine select version()
2021-11-09 16:13:00,057 INFO sqlalchemy.engine.Engine [generated in 0.00071s] ()
2021-11-09 16:13:00,144 INFO sqlalchemy.engine.Engine SELECT crdb_internal.increment_feature_counter(%s)
2021-11-09 16:13:00,145 INFO sqlalchemy.engine.Engine [generated in 0.00094s] ('sqlalchemy-cockroachdb 1.4.3.dev0',)
2021-11-09 16:13:00,148 INFO sqlalchemy.engine.Engine SELECT crdb_internal.increment_feature_counter(%s)
2021-11-09 16:13:00,149 INFO sqlalchemy.engine.Engine [cached since 0.004408s ago] ('sqlalchemy 2.0',)
2021-11-09 16:13:00,152 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-09 16:13:00,154 INFO sqlalchemy.engine.Engine SELECT table_name FROM information_schema.tables WHERE table_schema=%s
2021-11-09 16:13:00,154 INFO sqlalchemy.engine.Engine [generated in 0.00077s]

<IPython.core.display.Javascript object>

In [8]:
async with engine.begin() as conn:
    await conn.run_sync(Base.metadata.create_all)

2021-11-09 16:13:06,526 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-09 16:13:06,529 INFO sqlalchemy.engine.Engine SELECT table_name FROM information_schema.tables WHERE table_schema=%s
2021-11-09 16:13:06,530 INFO sqlalchemy.engine.Engine [cached since 6.377s ago] ('public',)
2021-11-09 16:13:06,539 INFO sqlalchemy.engine.Engine SELECT table_name FROM information_schema.tables WHERE table_schema=%s
2021-11-09 16:13:06,540 INFO sqlalchemy.engine.Engine [cached since 6.387s ago] ('public',)
2021-11-09 16:13:06,544 INFO sqlalchemy.engine.Engine 
CREATE TABLE "user" (
	id SERIAL, 
	name VARCHAR NOT NULL, 
	PRIMARY KEY (id)
)


2021-11-09 16:13:06,545 INFO sqlalchemy.engine.Engine [no key 0.00112s] ()
2021-11-09 16:13:06,954 INFO sqlalchemy.engine.Engine CREATE INDEX ix_user_name ON "user" (name)
2021-11-09 16:13:06,955 INFO sqlalchemy.engine.Engine [no key 0.00081s] ()
2021-11-09 16:13:07,321 INFO sqlalchemy.engine.Engine CREATE INDEX ix_user_id ON "user" (id)
2021-11-09 16:13:0

<IPython.core.display.Javascript object>

### Add data

Nick, Eli, and Paul sing 99 bottles of beer...

In [9]:
users = [User(name="Nick"), User(name="Eli"), User(name="Paul")]
users

[User(id=None, name='Nick'),
 User(id=None, name='Eli'),
 User(id=None, name='Paul')]

<IPython.core.display.Javascript object>

In [10]:
async with async_session() as session:
    session.add_all(users)
    await session.commit()

2021-11-09 16:13:13,486 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-09 16:13:13,489 INFO sqlalchemy.engine.Engine INSERT INTO "user" (name) VALUES (%s) RETURNING "user".id
2021-11-09 16:13:13,490 INFO sqlalchemy.engine.Engine [generated in 0.00107s] ('Nick',)
2021-11-09 16:13:15,437 INFO sqlalchemy.engine.Engine INSERT INTO "user" (name) VALUES (%s) RETURNING "user".id
2021-11-09 16:13:15,438 INFO sqlalchemy.engine.Engine [cached since 1.949s ago] ('Eli',)
2021-11-09 16:13:15,441 INFO sqlalchemy.engine.Engine INSERT INTO "user" (name) VALUES (%s) RETURNING "user".id
2021-11-09 16:13:15,442 INFO sqlalchemy.engine.Engine [cached since 1.953s ago] ('Paul',)
2021-11-09 16:13:15,446 INFO sqlalchemy.engine.Engine COMMIT


<IPython.core.display.Javascript object>

In [11]:
import itertools

cycle = itertools.cycle(users)
messages = []

for i in reversed(range(100)):
    content = f"{i} bottles of beer on the wall..."
    user = next(cycle)
    message = Message(user=user, content=content)
    messages.append(message)

async with async_session() as session:
    session.add_all(messages)
    await session.commit()

2021-11-09 16:13:16,141 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-09 16:13:16,146 INFO sqlalchemy.engine.Engine SELECT "user".id AS user_id, "user".name AS user_name 
FROM "user" 
WHERE "user".id = %s
2021-11-09 16:13:16,147 INFO sqlalchemy.engine.Engine [generated in 0.00119s] (709112610343288833,)
2021-11-09 16:13:17,728 INFO sqlalchemy.engine.Engine SELECT "user".id AS user_id, "user".name AS user_name 
FROM "user" 
WHERE "user".id = %s
2021-11-09 16:13:17,729 INFO sqlalchemy.engine.Engine [cached since 1.583s ago] (709112612259201025,)
2021-11-09 16:13:17,732 INFO sqlalchemy.engine.Engine SELECT "user".id AS user_id, "user".name AS user_name 
FROM "user" 
WHERE "user".id = %s
2021-11-09 16:13:17,733 INFO sqlalchemy.engine.Engine [cached since 1.587s ago] (709112612272340993,)
2021-11-09 16:13:17,740 INFO sqlalchemy.engine.Engine INSERT INTO message (content, user_id) VALUES (%s, %s) RETURNING message.id
2021-11-09 16:13:17,741 INFO sqlalchemy.engine.Engine [generated i

2021-11-09 16:13:18,625 INFO sqlalchemy.engine.Engine [cached since 0.886s ago] ('72 bottles of beer on the wall...', 709112610343288833)
2021-11-09 16:13:18,629 INFO sqlalchemy.engine.Engine INSERT INTO message (content, user_id) VALUES (%s, %s) RETURNING message.id
2021-11-09 16:13:18,630 INFO sqlalchemy.engine.Engine [cached since 0.8904s ago] ('71 bottles of beer on the wall...', 709112612259201025)
2021-11-09 16:13:18,633 INFO sqlalchemy.engine.Engine INSERT INTO message (content, user_id) VALUES (%s, %s) RETURNING message.id
2021-11-09 16:13:18,634 INFO sqlalchemy.engine.Engine [cached since 0.8945s ago] ('70 bottles of beer on the wall...', 709112612272340993)
2021-11-09 16:13:18,637 INFO sqlalchemy.engine.Engine INSERT INTO message (content, user_id) VALUES (%s, %s) RETURNING message.id
2021-11-09 16:13:18,638 INFO sqlalchemy.engine.Engine [cached since 0.8982s ago] ('69 bottles of beer on the wall...', 709112610343288833)
2021-11-09 16:13:18,642 INFO sqlalchemy.engine.Engine I

2021-11-09 16:13:18,769 INFO sqlalchemy.engine.Engine INSERT INTO message (content, user_id) VALUES (%s, %s) RETURNING message.id
2021-11-09 16:13:18,770 INFO sqlalchemy.engine.Engine [cached since 1.031s ago] ('41 bottles of beer on the wall...', 709112612259201025)
2021-11-09 16:13:18,775 INFO sqlalchemy.engine.Engine INSERT INTO message (content, user_id) VALUES (%s, %s) RETURNING message.id
2021-11-09 16:13:18,776 INFO sqlalchemy.engine.Engine [cached since 1.036s ago] ('40 bottles of beer on the wall...', 709112612272340993)
2021-11-09 16:13:18,779 INFO sqlalchemy.engine.Engine INSERT INTO message (content, user_id) VALUES (%s, %s) RETURNING message.id
2021-11-09 16:13:18,780 INFO sqlalchemy.engine.Engine [cached since 1.041s ago] ('39 bottles of beer on the wall...', 709112610343288833)
2021-11-09 16:13:18,783 INFO sqlalchemy.engine.Engine INSERT INTO message (content, user_id) VALUES (%s, %s) RETURNING message.id
2021-11-09 16:13:18,784 INFO sqlalchemy.engine.Engine [cached sinc

2021-11-09 16:13:18,899 INFO sqlalchemy.engine.Engine INSERT INTO message (content, user_id) VALUES (%s, %s) RETURNING message.id
2021-11-09 16:13:18,901 INFO sqlalchemy.engine.Engine [cached since 1.161s ago] ('10 bottles of beer on the wall...', 709112612272340993)
2021-11-09 16:13:18,904 INFO sqlalchemy.engine.Engine INSERT INTO message (content, user_id) VALUES (%s, %s) RETURNING message.id
2021-11-09 16:13:18,905 INFO sqlalchemy.engine.Engine [cached since 1.165s ago] ('9 bottles of beer on the wall...', 709112610343288833)
2021-11-09 16:13:18,907 INFO sqlalchemy.engine.Engine INSERT INTO message (content, user_id) VALUES (%s, %s) RETURNING message.id
2021-11-09 16:13:18,908 INFO sqlalchemy.engine.Engine [cached since 1.169s ago] ('8 bottles of beer on the wall...', 709112612259201025)
2021-11-09 16:13:18,911 INFO sqlalchemy.engine.Engine INSERT INTO message (content, user_id) VALUES (%s, %s) RETURNING message.id
2021-11-09 16:13:18,912 INFO sqlalchemy.engine.Engine [cached since 

<IPython.core.display.Javascript object>

### Query for data

#### Count queries

In [12]:
async with async_session() as session:
    # alternatively:
    # statement = sa.select(sa.func.count()).select_from(User)
    statement = sa.select(sa.func.count(User.id))
    results = await session.scalar(statement)


results

2021-11-09 16:13:19,524 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-09 16:13:19,527 INFO sqlalchemy.engine.Engine SELECT count("user".id) AS count_1 
FROM "user"
2021-11-09 16:13:19,528 INFO sqlalchemy.engine.Engine [generated in 0.00143s] ()
2021-11-09 16:13:19,535 INFO sqlalchemy.engine.Engine ROLLBACK


3

<IPython.core.display.Javascript object>

In [13]:
async with async_session() as session:
    statement = sa.select(sa.func.count(Message.id))
    results = await session.scalar(statement)

results

2021-11-09 16:13:20,301 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-09 16:13:20,304 INFO sqlalchemy.engine.Engine SELECT count(message.id) AS count_1 
FROM message
2021-11-09 16:13:20,305 INFO sqlalchemy.engine.Engine [generated in 0.00114s] ()
2021-11-09 16:13:20,361 INFO sqlalchemy.engine.Engine ROLLBACK


100

<IPython.core.display.Javascript object>

#### Query for user with messages

In [14]:
async with async_session() as session:
    statement = sa.select(User).options(sa.orm.joinedload(User.messages))
    results = await session.execute(statement)

results

2021-11-09 16:13:21,274 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-09 16:13:21,278 INFO sqlalchemy.engine.Engine SELECT "user".id, "user".name, message_1.id AS id_1, message_1.content, message_1.user_id 
FROM "user" LEFT OUTER JOIN message AS message_1 ON "user".id = message_1.user_id
2021-11-09 16:13:21,278 INFO sqlalchemy.engine.Engine [generated in 0.00070s] ()
2021-11-09 16:13:21,285 INFO sqlalchemy.engine.Engine ROLLBACK


<IPython.core.display.Javascript object>

In [15]:
users = results.unique().scalars().all()
users

[User(id=709112610343288833, name='Nick', messages=[Message(id=709112622307409921, user_id=709112610343288833, content='99 bottles of beer on the wall...'), Message(id=709112623735341057, user_id=709112610343288833, content='0 bottles of beer on the wall...'), Message(id=709112623694741505, user_id=709112610343288833, content='3 bottles of beer on the wall...'), Message(id=709112623654862849, user_id=709112610343288833, content='6 bottles of beer on the wall...'), Message(id=709112623617540097, user_id=709112610343288833, content='9 bottles of beer on the wall...'), Message(id=709112623577661441, user_id=709112610343288833, content='12 bottles of beer on the wall...'), Message(id=709112623538569217, user_id=709112610343288833, content='15 bottles of beer on the wall...'), Message(id=709112623495708673, user_id=709112610343288833, content='18 bottles of beer on the wall...'), Message(id=709112623452422145, user_id=709112610343288833, content='21 bottles of beer on the wall...'), Message

<IPython.core.display.Javascript object>

In [16]:
users[0].name

'Nick'

<IPython.core.display.Javascript object>

In [17]:
users[0].messages[:5]

[Message(id=709112622307409921, user_id=709112610343288833, content='99 bottles of beer on the wall...'),
 Message(id=709112623735341057, user_id=709112610343288833, content='0 bottles of beer on the wall...'),
 Message(id=709112623694741505, user_id=709112610343288833, content='3 bottles of beer on the wall...'),
 Message(id=709112623654862849, user_id=709112610343288833, content='6 bottles of beer on the wall...'),
 Message(id=709112623617540097, user_id=709112610343288833, content='9 bottles of beer on the wall...')]

<IPython.core.display.Javascript object>

In [18]:
for m in users[0].messages[:5]:
    print(m.content)

99 bottles of beer on the wall...
0 bottles of beer on the wall...
3 bottles of beer on the wall...
6 bottles of beer on the wall...
9 bottles of beer on the wall...


<IPython.core.display.Javascript object>

#### Chained statement example

In [19]:
async with async_session() as session:
    statement = sa.select(User)
    statement = statement.where(User.name == "Eli")
    statement = statement.options(sa.orm.selectinload(User.messages))
    results = await session.execute(statement)

user = results.scalars().first()
user

2021-11-09 16:13:25,760 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-09 16:13:25,762 INFO sqlalchemy.engine.Engine SELECT "user".id, "user".name 
FROM "user" 
WHERE "user".name = %s
2021-11-09 16:13:25,763 INFO sqlalchemy.engine.Engine [generated in 0.00095s] ('Eli',)
2021-11-09 16:13:25,770 INFO sqlalchemy.engine.Engine SELECT message.user_id AS message_user_id, message.id AS message_id, message.content AS message_content 
FROM message 
WHERE message.user_id IN (%s)
2021-11-09 16:13:25,771 INFO sqlalchemy.engine.Engine [generated in 0.00120s] (709112612259201025,)
2021-11-09 16:13:25,776 INFO sqlalchemy.engine.Engine ROLLBACK


User(id=709112612259201025, name='Eli', messages=[Message(id=709112622318911489, user_id=709112612259201025, content='98 bottles of beer on the wall...'), Message(id=709112622362787841, user_id=709112612259201025, content='95 bottles of beer on the wall...'), Message(id=709112622409220097, user_id=709112612259201025, content='92 bottles of beer on the wall...'), Message(id=709112622455554049, user_id=709112612259201025, content='89 bottles of beer on the wall...'), Message(id=709112622501462017, user_id=709112612259201025, content='86 bottles of beer on the wall...'), Message(id=709112622545076225, user_id=709112612259201025, content='83 bottles of beer on the wall...'), Message(id=709112622586396673, user_id=709112612259201025, content='80 bottles of beer on the wall...'), Message(id=709112622634434561, user_id=709112612259201025, content='77 bottles of beer on the wall...'), Message(id=709112622677131265, user_id=709112612259201025, content='74 bottles of beer on the wall...'), Messa

<IPython.core.display.Javascript object>

In [20]:
user.name

'Eli'

<IPython.core.display.Javascript object>

In [21]:
for m in user.messages[:5]:
    print(m.content)

98 bottles of beer on the wall...
95 bottles of beer on the wall...
92 bottles of beer on the wall...
89 bottles of beer on the wall...
86 bottles of beer on the wall...


<IPython.core.display.Javascript object>

### Using classmethod queries

In [22]:
async with async_session() as session:
    messages = await Message.afrom_user(session, "Eli")

len(messages)

2021-11-09 16:13:28,226 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-09 16:13:28,228 INFO sqlalchemy.engine.Engine SELECT message.id, message.content, message.user_id 
FROM message JOIN "user" ON "user".id = message.user_id 
WHERE "user".name = %s
2021-11-09 16:13:28,229 INFO sqlalchemy.engine.Engine [generated in 0.00102s] ('Eli',)
2021-11-09 16:13:28,236 INFO sqlalchemy.engine.Engine ROLLBACK


33

<IPython.core.display.Javascript object>

In [23]:
messages[:5]

[Message(id=709112622318911489, user_id=709112612259201025, content='98 bottles of beer on the wall...'),
 Message(id=709112622362787841, user_id=709112612259201025, content='95 bottles of beer on the wall...'),
 Message(id=709112622409220097, user_id=709112612259201025, content='92 bottles of beer on the wall...'),
 Message(id=709112622455554049, user_id=709112612259201025, content='89 bottles of beer on the wall...'),
 Message(id=709112622501462017, user_id=709112612259201025, content='86 bottles of beer on the wall...')]

<IPython.core.display.Javascript object>